Deck class is to handle which cards are in the deck. Preventing duplicated cards to appear.

In [222]:
# Deck class
import sys
import random

sys.setrecursionlimit(3000)

class Deck():
    deck=[]
    clubs=list(zip(['clubs']*13,range(1,14)))
    diamonds=list(zip(['diamonds']*13,range(1,14)))
    hearts=list(zip(['hearts']*13,range(1,14)))
    spades=list(zip(['spades']*13,range(1,14)))
    
    def __init__(self):
        self.shuffle()
        #self.deck= self.clubs + self.diamonds + self.hearts + self.spades
        #print(len(self.deck))
        pass
    
    def shuffle(self):
        self.deck= self.clubs + self.diamonds + self.hearts + self.spades
        random.shuffle(self.deck)
        print(len(self.deck))
        pass
    
    def draw(self):
        self.card=self.deck.pop(random.randint(0,(len(self.deck)-1)))
        print('deck remain',len(self.deck))
        return self.card
    
    def report(self):
        num=len(self.deck)
        print(f'There are {num} cards in the deck\n')
        ask=input('Show deck?\n')
        if ask.lower() in ['yes','y']:
            print('\nDeck :')
            print(self.deck)
            print('\n')
        else:
            print('\n')


In [160]:
deck=Deck()

52


In [161]:
deck.draw()

51


('clubs', 5)

PlayerHand class is to handle which cards aer in the player hand.

In [223]:
# Player hand class
class PlayerHand():
    hand=[]
    
    def __init__(self,owner='defult'):
        self.owner=owner
        self.clear()
        
    def hit(self,card):
        self.hand += [card]
        print(self.hand)
        
    def clear(self):
        self.hand=[]
        
    def report(self):
        num=len(self.hand)
        print(f"There are {num} cards in the {self.owner}'s hand")
        ask=input(f'Show {self.owner} hand?\n')
        if ask.lower() in ['yes','y']:
            print('\nHand :')
            print(self.hand)
            print('\n')
        else:
             print('\n')

In [163]:
playerhand=PlayerHand()

In [164]:
playerhand.hit(['dummy',2])

['dummy', 2]


GameController class is to handle the game mechanics.
Keep teacking the player's money
Game rounds
Dealer's rounds

In [224]:
#import Deck
#import PlayerHand
# Game class
class GameController():
    
    player_money=0
    game_state=False
    isplaying=True
    
    # Game setting, how many money dose the player have
    def __init__(self):
        self.game_setting()
        
    
    def game_setting(self):
        # Grettings
        print('Welcome to Black Jack game, how many money are you gaming?\n')
        while True:
            try:
                self.player_money=int(input('Please enter your money: '))
            except:
                print('You can not bet that, and we do not accept change.\n')
            else:
                print(f'Your bet {self.player_money} acceptted.')
                break
        self.game()
        
    def game(self):   
        
        isplaying=True
        
        while isplaying:
            
            ready=False
            
            while not ready:

                answer = input('Are you ready? (Yes or No)\n')

                if answer.lower() in ['yes','y']:
                    print('\nLet the game begine.\n')
                    ready=True
                    game_state=True
                elif answer.lower() in ['no','n']:
                    ready=False
                    print('OK, take your time.\n')
                else:
                    print('That is not an answer.\n')
            
            deck=Deck()            
            deck.report() # TODO delete
            
            player_hand=PlayerHand('player')
            player_hand.report() # TODO delete
            
            dealer_hand=PlayerHand('dealer')
            dealer_hand.report() # TODO delete
            
            turn='player'
            
            # Game start
            while game_state:
                if self.player_money<=0:
                    print('No more money left, bye')
                    isplaying=False
                    break
                else:
                    bet_ready=False
                                        
                    while not bet_ready:
                        try:
                            bet=int(input('Place your bet! :'))
                        except:
                            print('Seriously... that is not a bet.\n')
                        else:
                            bet_ready=True
                            
                    print(f"Your bet is : {bet}\n")
                
                # Player's turn
                while turn=='player':
                    
                    if len(player_hand.hand)==0:
                        player_hand.hit(deck.draw())
                        player_hand.report()
                    else:
                        ask = input('Hit or stand?')
                        if ask.lower()=='hit':
                            player_hand.hit(deck.draw())
                            player_hand.report()
                        elif ask.loer()=='stand':
                            player_hand.report()
                            turn='dealer'
                    
                    # Check for hand add up
                    add=0                    
                    for i,v in player_hand.hand:
                        if v>=11:
                            add+=10
                        elif v>=2:
                            add+=v
                        else:
                            if add+11 <= 21:
                                add+=11
                            elif add+1 <= 21:
                                add+=1
                    if add>21:
                        print("\nBusted!\n")
                    print(f'Your hand add up is {add}\n')
                    
           
                                

                    # stand or bust


                # Dealer's turn

                    # win or bust

                # Next round

                    # Player has money / game over

                # Playagin?

                break
            break

In [225]:
game=GameController()

Welcome to Black Jack game, how many money are you gaming?

Please enter your money: 555
Your bet 555 acceptted.
Are you ready? (Yes or No)
y

Let the game begine.

52
There are 52 cards in the deck

Show deck?
n


There are 0 cards in the player's hand
Show player hand?
n


There are 0 cards in the dealer's hand
Show dealer hand?
n


Place your bet! :5
Your bet is : 5

51
[('spades', 12)]
There are 1 cards in the player's hand
Show player hand?
y

Hand :
[('spades', 12)]


Your hand add up is 10

Hit or stand?hit
50
[('spades', 12), ('diamonds', 2)]
There are 2 cards in the player's hand
Show player hand?
y

Hand :
[('spades', 12), ('diamonds', 2)]


Your hand add up is 12

Hit or stand?hit
49
[('spades', 12), ('diamonds', 2), ('clubs', 13)]
There are 3 cards in the player's hand
Show player hand?
y

Hand :
[('spades', 12), ('diamonds', 2), ('clubs', 13)]


Your hand add up is 22

Hit or stand?


AttributeError: 'str' object has no attribute 'loer'

In [ ]:
game.player_money